In [1]:
import pandas as pd
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import random
from operator import itemgetter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [2]:
# Load Train data 
df_train_prepared = pd.read_pickle('df_train_prepared')

train_payprice = df_train_prepared['payprice']
train_bidprice = df_train_prepared['bidprice']
train_click = df_train_prepared['click']

df_train = df_train_prepared.drop(['click','payprice','bidprice'], axis =1)
df_train.head()



,weekday,hour,region,city,adexchange,slotwidth,slotheight,slotprice,advertiser,useragent_encoded,IP_encoded,url_encoded,domain_encoded,slotid_encoded,slotvisibility_encoded,slotformat_encoded,creative_encoded,keypage_encoded,usertag_encoded
0,5,22,2,2,2,200,200,5,3427,31,253587,268016,21783,12845,2,0,104,0,744035
1,1,20,238,239,1,300,250,0,2821,29,282720,145905,728,50771,6,3,9,18,744035
2,3,13,40,41,2,250,250,5,3427,31,469429,262017,18426,9136,2,0,92,0,188248
3,6,23,1,1,1,160,600,0,1458,31,107833,658015,4770,50450,2,1,111,12,706361
4,5,6,216,233,2,728,90,133,2259,31,342191,157670,153,1517,8,3,84,18,744035


In [3]:
# Scale validation data for better prediction when using logistic 
standard_scaler = StandardScaler().fit(df_train)
train_x = standard_scaler.transform(df_train)
train_x[:1]

array([[ 1.02735543,  1.2959677 , -1.44404095, -1.47450021, -0.07241839,
        -1.17330636,  0.08690655, -0.58626423,  0.74759644,  0.34754529,
        -0.02093817, -0.53713615,  1.42983954, -1.79876415, -0.03051079,
        -0.74630917,  0.83640481, -1.64992635,  1.21503638]])

In [4]:
# Load validation data
df_val_prepared = pd.read_pickle('df_val_prepared')

val_payprice = df_val_prepared['payprice']
val_bidprice = df_val_prepared['bidprice']
val_click = df_val_prepared['click']

df_val = df_val_prepared.drop(['click','payprice','bidprice'], axis =1)
df_val.head()

,weekday,hour,region,city,adexchange,slotwidth,slotheight,slotprice,advertiser,useragent_encoded,IP_encoded,url_encoded,domain_encoded,slotid_encoded,slotvisibility_encoded,slotformat_encoded,creative_encoded,keypage_encoded,usertag_encoded
0,4,20,79,79,1,160,600,0,1458,28,137474,45239,8595,14275,2,1,109,12,110919
1,1,21,79,79,1,950,90,0,3476,26,178167,109524,7201,14355,0,1,108,7,69995
2,4,8,2,2,2,300,250,5,3358,28,52834,75356,8455,7129,2,0,99,14,108448
3,5,15,201,205,2,336,280,5,3358,26,192739,27901,8931,8936,2,0,116,14,109043
4,1,18,134,135,2,200,200,5,3476,26,55070,135142,6582,423,1,0,42,5,6344


In [5]:
# Scale validation data for better prediction when using logistic 
standard_scaler = StandardScaler().fit(df_val)
test_x = standard_scaler.transform(df_val)
test_x[:1]

array([[ 0.54310061,  0.98980048, -0.68871428, -0.72819482, -1.16598138,
        -1.30074929,  3.12709522, -0.72222819, -1.75178669,  0.35195772,
         0.67377474, -0.60695559,  1.33457283,  0.58957062, -0.02894579,
         0.11112068,  1.01082565,  0.28468824,  1.09400445]])

In [6]:
val_click[891]

1

### Average CTR for Validation

In [7]:
val_click_1 = df_val_prepared[df_val_prepared['click'] == 1]
val_AvgCTR =  float("{0:.5f}".format(len(val_click_1)/len(df_val_prepared['click'])))
val_AvgCTR 

0.00066

### Logistic Regression to Predict Probability of Click or Not Click

In [8]:
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix 
 
log_reg = LogisticRegression(C = 100000, class_weight='balanced',random_state = 0)
clf1 = linear_model.LogisticRegression()
clf1.fit(train_x, train_click)
log1_prob = clf1.predict_proba(test_x)

# Add threshold D = 0.0007
df_log1 = pd.DataFrame(columns=['probability_0_click','probability_1_click'], data = log1_prob)
log1_pred = np.where(df_log1['probability_1_click'] <= 0.00058,1,0)

log1_error = sum(log_click != val_click) 
log1_correct = sum(log_click == val_click) 
print('Errors from logistic:' ,errors)

confusion_matrix(val_click, log1_pred)

NameError: name 'log_click' is not defined

### LOGISTIC PREDICTION USING UNDER SAMPLING 

- Data is way too imblance only very little of click -> large prediction error
- Solution: UNDER SAMPLE
- Balance the click and non click -> improve the prediction for click or non click
 

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

mean_class_size = int(pd.Series(train_click).value_counts().sum()/1500)
mean_class_size

ratio = {0: mean_class_size,
         1: mean_class_size}

rus = RandomUnderSampler (ratio = ratio, random_state = 0)
rus

train_x_rus, train_y_rus = rus.fit_sample(train_x, train_click)

In [ ]:
log_reg = LogisticRegression()
clf2 = linear_model.LogisticRegression()
clf2.fit(train_x_rus, train_y_rus)

log2_prob  = clf2.predict_proba(test_x)
log2_pred = clf2.predict(test_x)

log2_error = sum(log2_pred!= val_click) 
log2_correct = sum(log_click == val_click) 
print('Errors from Logistic with Under Sample:', log2_errors)

confusion_matrix(val_click,log2_pred)


In [ ]:
df_log2 = pd.DataFrame(columns=['probability_0_click','probability_1_click'], data = log2_prob)
df_log2['click_or_noclick'] = log2_pred
df_logistic[840:845]

In [ ]:
# We can use the probability as pCTR per row 
# or we can calculate pCRT from this prediction 
click_log2 = len(df_log2[df_log2['click_or_noclick'] == 1])

pCTR_log = click_log2/ len(df_log2)
pCTR_log

### RANDOM FOREST

Random Forest produce slightly better prediction

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(train_x_rus, train_y_rus)

In [ ]:
tree_pred = tree_clf.predict(test_x)
tree_prob = tree_clf.predict_proba(test_x)

tree_error = sum(tree_pred != val_click) 
tree_correct = sum(tree_pred == val_click) 
print('Errors from Decision Tree with Under Sample:', tree_errors)

confusion_matrix(val_click, tree_pred)

In [ ]:
# Put in df
df_tree_predicted = pd.DataFrame(columns=['probability_0_click','probability_1_click'], data = tree_prob)

df_tree_predicted['click_or_noclick'] = tree_pred 
df_tree_predicted.to_pickle('df_tree_prediction')
df_tree_predicted[840:845]

In [ ]:
# We can use the probability as pCTR per row 
# or we can calculate pCRT from this prediction 

click1_tree = len(df_tree[df_tree['click_or_noclick'] == 1])
pCTR_tree = click1_tree/ len(df_tree)
pCTR_tree

In [ ]:
# Mean square error for three models
#log1_rmse = np.sqrt(mean_squared_error(val_click, log1_pred))

log1_rmse = float("{0:.2f}".format(np.sqrt(mean_squared_error(val_click, log1_pred))))
log2_rmse = float("{0:.2f}".format(np.sqrt(mean_squared_error(val_click, log2_pred))))
tree_rmse = float("{0:.2f}".format(np.sqrt(mean_squared_error(val_click, tree_pred))))


model_errors = {'Logistic Regression': [log1_rmse, log1_error, log1_correct] , 'Logistic with Under Sample': [log2_rmse, log2_error, log2_correct], 'Decision Tree': [tree_rmse,tree_errors, tree_correct]}
labels = ['RMSE', 'Misclassified', 'Correct Classified']

df_error = pd.DataFrame(model_errors, index = labels)
df_error



### Linear Bidding 

basebid * pCRT/averageCTR 

pCTR = probability of click from Decision Tree Model 

In [ ]:
# Standard Deviation of Bid Price 

std = np.std(val_bidprice)/1000
std 

base= val_bidprice.mean()/1000 + (2*std )
base = float("{0:.2f}".format(base))
base 

In [ ]:
df_tree_predicted